# III. Precomputing your favorite PBH accretion emission model (WIP)

In 2509.22772, we explored a set of PBH accretion models as variations to base models $\textbf{PR}$ and $\textbf{BHL}$, based on Park and Ricotti (2013) and the Bondi-Hoyle-Lyttleton accretion model respectively, with the emission mechanism based on the hot Advection-Dominated Accretion Flow (ADAF) model. We have demonstrated that model variations to both the accretion and the emission model can impact the modification to the $T_{21}$ power spectrum, and hence the projected sensitivity for HERA. Here, we provide the reader with a guide to run `DM21cm` with alternative modeling choices to further explore the modeling variations.

# 1. Modify existing `PBHAccretionModel` options

In [ ]:
%reload_ext autoreload
%autoreload 2

from dm21cm.precompute.accretion import PBHAccretionModel

The options for `PBHAccretionModel` can be viewed in the docstring:

In [2]:
PBHAccretionModel?

Init signature:
PBHAccretionModel(
    model,
    c_in=23,
    lambda_fudge=1,
    v_rel_type='DMRest',
    delta_e=0.1,
)
Docstring:      <no docstring>
Init docstring:
PBH accretion emission model.

Args:
    model (str): supports 'PR-ADAF', 'BHL-ADAF', 'PRHALO-ADAF'.
    c_in (float): Sound speed inside the I-front [km/s] in PR models. Required for PR models.
    lambda_fudge (float): Fudge factor in front of Mdot. Required for BHL models.
    v_rel_type {'DMDM', 'DMRest'}: Velocity distribution option.
        'DMDM'   - DM-DM relative velocity distribution function.
        'DMRest' - DM-rest frame relative velocity distribution function (default).
    delta_e (float): ADAF electron heating fraction. Default is 0.1.
File:           ~/dm21cm/DM21cm/src/dm21cm/precompute/accretion.py
Type:           type
Subclasses:     

For example, we can combine the effect of a decreased interior sound speed $c_\text{in}$, and an increased electron heating fraction $\delta_e$ (i.e. combining $\textbf{PR}$-$c_\text{in}^-$ and $\textbf{PR}$-$\delta_e^+$, see text in Sec. III of 2509.22772), by doing:

In [3]:
am = PBHAccretionModel(model='PR-ADAF', c_in=14, delta_e=0.5)

One can then put this line in the `===== Initialization =====` section of [build_pbhacc_tables.py](../src/dm21cm/precompute/scripts/build_pbhacc_tables.py), and run the script.

# 2. Defining a fully custom model

A custom model only requires the following functions `L_halo`, `L_cosmo_density`, `L_cosmo_density_vcbavg` to be used in [build_pbhacc_tables.py](../src/dm21cm/precompute/scripts/build_pbhacc_tables.py). The script vectorizes some of this functions itself. Use `jnp` in place of `np` for mathematical operations.

In [4]:
import jax.numpy as jnp

class MyPBHAccretionModel:

    def L_halo(self, m_PBH, m_halo, c_halo, z):
        """PBH accretion luminosity in a halo [M_sun/yr]
        
        Args:
            m_PBH (float): Mass of the PBH [M_sun]
            m_halo (float): Mass of the halo [M_sun]
            c_halo (float): Halo concentration parameter
            z (float): Redshift
        """
        return jnp.sin(m_PBH)**2 + jnp.cos(m_halo)**2 # dummy example

    def L_cosmo_density(self, m_PBH, z, rho_dm, rho_b_inf, c_inf, v_cb):
        """PBH accretion luminosity conformal density [M_sun/yr/cMpc^3]
        
        Args:
            z (float): Redshift
            rho_dm (float): DM density (need to account for f_collapsed) [M_sun/cMpc^3]
            rho_b_inf (float): Ambient gas density [g/cm^3]
            c_inf (float): Ambient gas sound speed [km/s]
            v_cb (float): DM-baryon streaming velocity [km/s]
        """
        return 0.

    def L_cosmo_density_vcbavg(self, m_PBH, z, rho_dm, rho_b_inf, c_inf):
        """PBH accretion luminosity conformal density [M_sun/yr/cMpc^3] vcb averaged
        
        Args:
            z (float): Redshift
            rho_dm (float): DM density (need to account for f_collapsed) [M_sun/cMpc^3]
            rho_b_inf (float): Ambient gas density [g/cm^3]
            c_inf (float): Ambient gas sound speed [km/s]
        """
        return 0.